In [3]:
from game import Game
import time
import os
import re
import heapq
from queue import Queue
from collections import deque

## Load maps

In [4]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [5]:
print("This is an example of the game map:")
map = load_map(3)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	W	.	.	W	W
W	H	W	.	B2	W	W
W	.	W	.	.	W	W
W	G2	W	B1	P1	W	W
W	W	W	.	W	W	W
W	W	W	G1	W	W	W
W	W	W	W	W	W	W


In [6]:
game.get_box_locations()

[(3, 3), (1, 4)]

In [7]:
game.get_goal_locations()

[(5, 3), (3, 1)]

In [8]:
game.get_player_position()

(1, 1)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [9]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: True
W	P1	W	.	.	W	W
W	.	W	.	B2	W	W
W	.	W	.	H	W	W
W	G2	W	B1	P1	W	W
W	W	W	.	W	W	W
W	W	W	G1	W	W	W
W	W	W	W	W	W	W
Move D is valid: True
W	P1	W	.	.	W	W
W	H	W	.	B2	W	W
W	.	W	.	.	W	W
W	G2	W	B1	P1	W	W
W	W	W	.	W	W	W
W	W	W	G1	W	W	W
W	W	W	W	W	W	W
Move R is valid: False
Move L is valid: False


In [10]:
print(game.is_move_valid('R'))
game.apply_move('D')
game.display_map()

False
W	P1	W	.	.	W	W
W	.	W	.	B2	W	W
W	H	W	.	.	W	W
W	G2	W	B1	P1	W	W
W	W	W	.	W	W	W
W	W	W	G1	W	W	W
W	W	W	W	W	W	W


In [11]:
if game.apply_moves(['L', 'U', 'D', 'D', 'U', 'L']): 
    game.display_map()
game.apply_moves(['R','U'])
print("Is game won?", game.is_game_won())

W	P1	W	.	.	W	W
W	.	W	.	B2	W	W
W	H	W	.	.	W	W
W	G2	W	B1	P1	W	W
W	W	W	.	W	W	W
W	W	W	G1	W	W	W
W	W	W	W	W	W	W
Is game won? False


## Solvers

### BFS

In [12]:
# TODO: Must return moves (if there is no solution return None), number of visited states
# for bfs , it is gauranteed that the answer we reach is the most optimal solution
from collections import deque

def solver_bfs(game_map_str):
    game = Game(game_map_str)
    
    # The state is (player_position, (box1_position, box2_position, ...))
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    visited_states = set([initial_state])
    # Each queue element is a tuple: (move_sequence, state)
    queue = deque([[]])
    
    while queue:
    
        path = queue.popleft()
       
        # Reset game to the initial state and apply the moves to reach the current state
        game.reset_state(initial_state)
        if path:
            game.apply_moves(path)
            
        current_state = (game.get_player_position(), tuple(game.get_box_locations()))
        if game.is_game_won():
            # Return the first (shortest) solution and number of visited states
            return path, len(visited_states)
        
        # Try all possible moves from the current state
        for move in ['U', 'R', 'D', 'L']:
            # Reset to the current state before testing the move
            game.reset_state(current_state)
            if game.apply_move(move):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()))
                if new_state not in visited_states:
                    visited_states.add(new_state)
                    new_path = path + [move]
                    queue.append(new_path)

    return None, len(visited_states)

map = load_map(10)
solver_bfs(map)

KeyboardInterrupt: 

### DFS

In [ ]:
# TODO: Must return moves, number of visited states
# for dfs, we can't be sure that the first solution we reach would be the most optimal one, 

from collections import deque
# my main implementation of dfs :
def solver_dfs(game_map):
    game = Game(game_map)

    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    visited_states = set([initial_state])

    stack = deque([[]]) 

    solutions = set()  

    while stack:
        path = stack.pop() 

        game.reset_state(initial_state)
        if path:
            game.apply_moves(path)

        current_state = (game.get_player_position(), tuple(game.get_box_locations()))

        if game.is_game_won():
            return path, len(visited_states)
     
        if current_state not in visited_states:
            visited_states.add(current_state)

        for direction in ['L','U','R','D']:
            game.reset_state(current_state)
            if game.apply_move(direction):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()))

                if new_state not in visited_states:
                    stack.append(path + [direction])
            
    return [], len(visited_states) 

map = load_map(8)
solver_dfs(map)

we're gonna implement something close to that dfs, but with the difference that we're gonna set a limit each time, and if we didn't reach the solution, we're gonna expand(exceed) that limit
but would that limit be ? the length of the path list, if the length of that limit reached a limit, turn back
but would that turn back be ?
turn back with an error ?
how to implement that turn back? 
-while starts and we get our right most path and turn90degree errors
-if the path size is more than limit, pop last move and increase error and push back and continue
-then if the turndegreee is above 3, pop the last element of that path, and push back this tuple : (decreased_length_path, error+1) and continue
-then we reset the game and apply the path, and get the current state
-if we won, return the path
-if the current state is not in the visited states add to it

### IDS

In [ ]:
# TODO: Must return moves, number of visited states

def solver_ids(game_map):
    game = Game(game_map)

    layer_limit = 2

    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    visited_states = set([initial_state])

    stack = deque([[]]) 

    solutions = set() 

    while stack:

        path = stack.pop() 

        game.reset_state(initial_state)
        if path:
            game.apply_moves(path)

        current_state = (game.get_player_position(), tuple(game.get_box_locations()))

        if game.is_game_won():
            return path, len(visited_states)

        if current_state not in visited_states:
            visited_states.add(current_state)

        for direction in ['L','U','R','D']:
            game.reset_state(current_state)
            if game.apply_move(direction):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()))

                if new_state not in visited_states and len(path) < layer_limit:
                    stack.append(path + [direction])
        
        if not stack:  
            layer_limit += 1
            visited_states.clear()
            stack = deque([[]])
            
    return [], len(visited_states) 

map = load_map(9)
print(solver_ids(map))

KeyboardInterrupt: 

### A*

In [ ]:
# TODO
import math
def heuristic(game): # heuristic is a function that estimates how close a state is to a goal
    '''
    #  the Eucidean distance 
    box_locations = game.get_box_locations()
    goal_locations = game.get_goal_locations()
    h = 0
    for i in range (0,len(box_locations)) :
        distances = math.sqrt(pow((box_locations[i][0] - goal_locations[i][0]), 2) + pow((box_locations[i][1] - goal_locations[i][1]), 2))
        h += distances
    return h
    '''
    # the manhattan distance
    box_locations = game.get_box_locations()
    goal_locations = game.get_goal_locations()
    h = 0
    for i in range (0,len(box_locations)) :
        distances = abs(box_locations[i][0] - goal_locations[i][0]) + abs(box_locations[i][1] - goal_locations[i][1])
        h += distances
    return h
    
    
def improved_heuristic(game):

    box_locations = game.get_box_locations()
    goal_locations = game.get_goal_locations()
    total_cost = 0
    deadlock_penalty = 1000  # Large penalty for deadlock situations

    for i in range(len(box_locations)):
        box = box_locations[i]
        goal = goal_locations[i]
        
        # Base cost: Manhattan distance for the paired box and goal.
        cost = abs(box[0] - goal[0]) + abs(box[1] - goal[1])
        
        # Only add extra penalties if the box is not yet on its goal.
        if box != goal:
            
            up    = (box[0] - 1, box[1])
            down  = (box[0] + 1, box[1])
            left  = (box[0], box[1] - 1)
            right = (box[0], box[1] + 1)
        
            if (game.is_wall(up) and game.is_wall(left)) or \
               (game.is_wall(up) and game.is_wall(right)) or \
               (game.is_wall(down) and game.is_wall(left)) or \
               (game.is_wall(down) and game.is_wall(right)):
                cost += deadlock_penalty

            if game.is_wall(left) and game.is_wall(right) and box[0] != goal[0]:
                cost += 5 * abs(box[0] - goal[0])
            
            if game.is_wall(up) and game.is_wall(down) and box[1] != goal[1]:
                cost += 5 * abs(box[1] - goal[1])
        
        total_cost += cost
    return total_cost


# TODO: Must return moves, number of visited states
def solver_astar(game_map, heuristic_func=improved_heuristic, weight=1.7):
    game = Game(game_map)

    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    visited_states = set([initial_state])

    h = heuristic_func(game)
    g = 0
    heap = [] 
    heapq.heappush(heap, (h + g, []))

    solutions = set()

    while heap:

        f, path = heapq.heappop(heap)

        game.reset_state(initial_state)
        if path:
            game.apply_moves(path)

        current_state = (game.get_player_position(), tuple(game.get_box_locations()))

        if game.is_game_won():
            return path, len(visited_states)

        if current_state not in visited_states:
            visited_states.add(current_state)

        for direction in ['L','U','R','D']:
            game.reset_state(current_state)
            if game.apply_move(direction):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()))

                if new_state not in visited_states:
                    g = len(path) + 1
                    h = heuristic_func(game)
                    heapq.heappush(heap, (g + weight*h,path + [direction]))
            
    return [], len(visited_states) 

map = load_map(10)
print(solver_astar(map))

(['R', 'R', 'R', 'R', 'R', 'D', 'R', 'U', 'L', 'U', 'R', 'U', 'L', 'L', 'L', 'D', 'L', 'L', 'U', 'R', 'U', 'U', 'L', 'D', 'R', 'D', 'L', 'D', 'R', 'R', 'D', 'R', 'U', 'L', 'U', 'R', 'U', 'R', 'D', 'D', 'R', 'D', 'L', 'L', 'L', 'L'], 12579)


## Solve

In [ ]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar
}

In [ ]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [ ]:
solve(1, "BFS") # Solve map 1 using BFS

BFS took 0.0 seconds on map 1 and visited 47 states.
7 moves were used: ['U', 'D', 'R', 'L', 'D', 'U', 'L']


In [ ]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            solve(map, method)
            

In [ ]:
solve_all() # Solve all maps using all methods

BFS took 0.0 seconds on map 1 and visited 47 states.
7 moves were used: ['U', 'D', 'R', 'L', 'D', 'U', 'L']
DFS took 0.0 seconds on map 1 and visited 10 states.
7 moves were used: ['D', 'U', 'R', 'L', 'U', 'D', 'L']
IDS took 0.0 seconds on map 1 and visited 10 states.
7 moves were used: ['D', 'U', 'R', 'L', 'U', 'D', 'L']


NameError: name 'stack' is not defined